In [2]:
import pandas as pd
from matplotlib import pyplot

In [ ]:
# bringing in data from NYC Open Data

# all the pools
pools = pd.read_csv('data/NYC_Parks_Pools_20250924.csv')

# pool session information, including problems and incidents
sessions = pd.read_csv('data/Outdoor_Pools_Session_Information_20250924.csv')

# contains pool dimensions in feet
outdoor = pd.read_json('data/DPR_Pools_outdoor_001.json')

In [4]:
print('pools', pools.shape)
print('sessions', sessions.shape)
print('outdoor', outdoor.shape)

pools (90, 11)
sessions (22308, 35)
outdoor (79, 11)


In [5]:
pools.head(1)

,BOROUGH,COMMUNITYBOARD,COUNCILDISTRICT,PARKDISTRICT,GISPROPNUM,LOCATION,NAME,OMPPROPID,SYSTEM,polygon,POOLTYPE
0,X,202,17,X-02,X307,Outdoor,Floating Pool,X307,X307-POOL-0028,POLYGON ((-73.8889859664428 40.804149906408306...,Intermediate


In [6]:
# X - Bronx
# B - Brooklyn
# M - Manhattan
# Q - Queens
# R - Staten Island

In [7]:
outdoor.head()

# Prop_ID: first character = borough

,Prop_ID,Name,Location,Phone,Pools_outdoor_Type,Setting,Size,Accessible,lat,lon,rec_center_id
0,X008,Claremont,170th Street and Clay Avenue,(718) 901-4792,Intermediate,Outdoor,75' x 60' x 3.5',Y,40.8390,-73.9073,None
1,X010,Crotona Pool,173rd Street and Fulton Avenue,(718) 731-6872,Olympic,Outdoor,330' x 120' x 4',Y,40.8399,-73.8981,X010
2,X196,Haffen,Ely and Burke Avenues,(718) 379-2908,Intermediate,Outdoor,75' x 60' x 3.5',Y,40.8743,-73.8392,None
3,X289,Mapes Pool,E. 180th Street between Mapes &amp; Prospect A...,(718) 364-8876,Intermediate,Outdoor,75' x 60' x 3.5',Y,40.8465,-73.8860,None
4,X034,Foster Pool,E. 164th St. bet. Jerome &amp; River Aves.,(646) 678-1938,Intermediate,Outdoor,75' x 60' x 3.5',Y,40.8317,-73.9243,X034


In [8]:
# Outdoor pools session information

# usually two sessions, sometimes a third extended one
# LateOpen column: TRUE indicates there might have been a poop incident
# LateOpenReason column: includes Category 1 (45 minute closure)
# ClosedReason column: pool session didn't open, includes Category 2 (25.5 hour closure)

pd.set_option('display.max_columns', None)
sessions.head()

# with pd.option_context('display.max_columns', None):
#     sessions.head()
# sessions.head()

,ObservationDate,Borough,Pool,PoolID,SessionType,OpenToday,ClosedReason,OpenTime,LateOpen,LateOpenReason,PoolOpenPercent,Attendance,SupervisorCount,LifeguardCount,PEPCount,NYPDCount,MeetingTime,Incident,IncidentCount,IncidentReport,Problem,ProblemCount,Arrest,ArrestCount,ChlorineType,LiquidChlorine,TabletChlorine,OvernightSwimmers,OvernightSwimmersCheck,OvernightSwimmerNum,PressYes,Breakfasts,Lunches,Temperature,WeatherConditions
0,28/06/2022,B,Lindower Park Mini Pool,B329,First,True,NaN,06/28/2022 11:00:00 AM,False,NaN,100.0,34.0,NaN,2.0,1.0,NaN,NaN,False,NaN,NaN,False,NaN,False,NaN,Tablets,NaN,100.0,NaN,NaN,0.0,False,0.0,0.0,Low 70s,Sunny
1,28/06/2022,R,Faber Pool,R008,First,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low 70s,Sunny
2,28/06/2022,M,Sheltering Arms Pool,M190,First,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low 70s,Sunny
3,28/06/2022,R,General Douglas MacArthur Mini Pool,R067,First,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High 70s,Sunny
4,28/06/2022,R,PS 14 Playground Mini Pool,R061,First,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mid 70s,Sunny


In [9]:
len(sessions.loc[sessions.LateOpenReason.notnull()])

1397

In [10]:
sessions.LateOpen.describe()

count     20784
unique        2
top       False
freq      19245
Name: LateOpen, dtype: object

In [13]:
sessions.Problem.value_counts()

Problem
False    20338
True      1094
Name: count, dtype: int64

In [14]:
sessions.SessionType.value_counts()

SessionType
First              10833
Second             10825
Extended Second      650
Name: count, dtype: int64

In [24]:
# df copy for cleaning
pools_clean = pools.copy()

# making a borough column with names instead of ids
borough_mapping = {
    'X': 'Bronx',
    'B': 'Brooklyn', 
    'M': 'Manhattan',
    'Q': 'Queens',
    'R': 'Staten Island'
}

pools_clean['Borough_Name'] = pools_clean['BOROUGH'].map(borough_mapping)

# name cleanup
pools_clean['LOCATION'] = pools_clean['LOCATION'].str.strip()
pools_clean['NAME'] = pools_clean['NAME'].str.strip()

# rename column
pools_clean.rename(columns={'OMPPROPID': 'POOL_ID'}, inplace=True)

print(f'shape: {pools_clean.shape}')
print(f'\n{pools_clean['Borough_Name'].value_counts()}')
print(f'\n{pools_clean['POOLTYPE'].value_counts()}')


shape: (90, 12)

Borough_Name
Manhattan        31
Brooklyn         23
Bronx            15
Queens           12
Staten Island     9
Name: count, dtype: int64

POOLTYPE
Intermediate             27
Wading                   23
Mini                     17
Olympic                  14
Large                     5
Diving                    2
Intermediate & Diving     1
Olympic & Diving          1
Name: count, dtype: int64


In [25]:
pools_clean.head()

,BOROUGH,COMMUNITYBOARD,COUNCILDISTRICT,PARKDISTRICT,GISPROPNUM,LOCATION,NAME,POOL_ID,SYSTEM,polygon,POOLTYPE,Borough_Name
0,X,202,17,X-02,X307,Outdoor,Floating Pool,X307,X307-POOL-0028,POLYGON ((-73.8889859664428 40.804149906408306...,Intermediate,Bronx
1,X,206,15,X-06,X236,Outdoor,Mapes Wading Pool,X236,X236-POOL-0027,POLYGON ((-73.88626280458823 40.84670312461004...,Wading,Bronx
2,X,206,15,X-06,X236,Outdoor,Mapes Pool,X236,X236-POOL-0026,POLYGON ((-73.88620760448572 40.84645972094962...,Intermediate,Bronx
3,X,212,12,X-12,X196,Outdoor,Haffen Pool,X196,X196-POOL-0025,"POLYGON ((-73.83937539674365 40.8743013365522,...",Intermediate,Bronx
4,X,212,12,X-12,X196,Outdoor,Haffen Wading Pool,X196,X196-POOL-0024,POLYGON ((-73.83942993261637 40.87444743038688...,Wading,Bronx


In [37]:
# df copy for cleaning
sessions_clean = sessions.copy()

# clean up date/time columns
sessions_clean['ObservationDate'] = pd.to_datetime(sessions_clean['ObservationDate'], format='%d/%m/%Y')
sessions_clean['OpenTime'] = pd.to_datetime(sessions_clean['OpenTime'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
# probably don't need this one
sessions_clean['MeetingTime'] = pd.to_datetime(sessions_clean['MeetingTime'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

# re-map borough column with boroughnames instead of ids 
sessions_clean['Borough_Name'] = sessions_clean['Borough'].map(borough_mapping)

# rename pool id column to match pools_clean
pools_clean.rename(columns={'PoolID': 'POOL_ID'}, inplace=True)

print(f"shape: {sessions_clean.shape}")
print(f"\ndate range: {sessions_clean['ObservationDate'].min()} to {sessions_clean['ObservationDate'].max()}\n\n")
print("number of swim sessions:")
print(f"\n{sessions_clean['Borough_Name'].value_counts()}")
print(f"\n{sessions_clean['SessionType'].value_counts()}")


shape: (22308, 36)

date range: 2022-06-28 00:00:00 to 2024-09-08 00:00:00


number of swim sessions:

Borough_Name
Brooklyn         6264
Manhattan        6047
Bronx            3932
Staten Island    3111
Queens           2954
Name: count, dtype: int64

SessionType
First              10833
Second             10825
Extended Second      650
Name: count, dtype: int64


In [28]:
sessions_clean.head()

,ObservationDate,Borough,Pool,PoolID,SessionType,OpenToday,ClosedReason,OpenTime,LateOpen,LateOpenReason,PoolOpenPercent,Attendance,SupervisorCount,LifeguardCount,PEPCount,NYPDCount,MeetingTime,Incident,IncidentCount,IncidentReport,Problem,ProblemCount,Arrest,ArrestCount,ChlorineType,LiquidChlorine,TabletChlorine,OvernightSwimmers,OvernightSwimmersCheck,OvernightSwimmerNum,PressYes,Breakfasts,Lunches,Temperature,WeatherConditions,Borough_Name
0,2022-06-28,B,Lindower Park Mini Pool,B329,First,True,nan,2022-06-28 11:00:00,False,nan,100.0,34.0,NaN,2.0,1.0,NaN,NaT,False,NaN,NaN,False,NaN,False,NaN,Tablets,NaN,100.0,NaN,NaN,0.0,False,0.0,0.0,Low 70s,Sunny,Brooklyn
1,2022-06-28,R,Faber Pool,R008,First,False,nan,NaT,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low 70s,Sunny,Staten Island
2,2022-06-28,M,Sheltering Arms Pool,M190,First,False,nan,NaT,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low 70s,Sunny,Manhattan
3,2022-06-28,R,General Douglas MacArthur Mini Pool,R067,First,False,nan,NaT,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High 70s,Sunny,Staten Island
4,2022-06-28,R,PS 14 Playground Mini Pool,R061,First,False,nan,NaT,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mid 70s,Sunny,Staten Island


In [31]:
# df copy for cleaning
outdoor_clean = outdoor.copy()

# Clean text columns
text_columns = ['Prop_ID', 'Name', 'Location', 'Phone', 'Pools_outdoor_Type', 'Setting', 'Size', 'Accessible']
for col in text_columns:
    if col in outdoor_clean.columns:
        outdoor_clean[col] = outdoor_clean[col].astype(str).str.strip()

# get borough from first character of Prop_ID
outdoor_clean['Borough_Code'] = outdoor_clean['Prop_ID'].str[0]
borough_mapping = {
    'X': 'Bronx',
    'B': 'Brooklyn', 
    'M': 'Manhattan',
    'Q': 'Queens',
    'R': 'Staten Island'
}
outdoor_clean['Borough_Name'] = outdoor_clean['Borough_Code'].map(borough_mapping)

# rename pool type column to match other dfs
outdoor_clean.rename(columns={'Pools_outdoor_Type': 'POOLTYPE'}, inplace=True)

print(f"shape: {outdoor_clean.shape}")
print(f"\n{outdoor_clean['Borough_Name'].value_counts()}")
print(f"\n{outdoor_clean['POOLTYPE'].value_counts()}")


shape: (79, 13)

Borough_Name
Manhattan        25
Brooklyn         20
Bronx            14
Queens           11
Staten Island     9
Name: count, dtype: int64

POOLTYPE
Wading          23
Intermediate    22
Mini            17
Olympic         14
Diving           3
Name: count, dtype: int64


In [57]:
# Analyze pool closure patterns
print("=== POOL CLOSURES ===")

# two types of closures, late open and not open at all
closed_pools = sessions_clean[sessions_clean['OpenToday'] == False]
late_open_pools = sessions_clean[sessions_clean['LateOpen'] == True]

print(f"total sessions: {len(sessions_clean):,}")
print(f"sessions where pools were closed: {len(closed_pools):,} ({len(closed_pools)/len(sessions_clean)*100:.1f}%)")
print(f"sessions with late openings: {len(late_open_pools):,} ({len(late_open_pools)/len(sessions_clean)*100:.1f}%)")

# closure reasons
print(f"\nClosure reasons:")
print(closed_pools['ClosedReason'].value_counts().head(30))

# late opening reasons
print(f"\nLate opening reasons:")
print(late_open_pools['LateOpenReason'].value_counts().head(30))

# Analyze by borough
print(f"\nClosure rate by borough:")
closure_by_borough = sessions_clean.groupby('Borough_Name').agg({
    'OpenToday': ['count', lambda x: (x == False).sum()]
}).round(2)
closure_by_borough.columns = ['Total_Sessions', 'Closed_Sessions']
closure_by_borough['Closure_Rate'] = (closure_by_borough['Closed_Sessions'] / closure_by_borough['Total_Sessions'] * 100).round(1)
print(closure_by_borough)


=== POOL CLOSURES ===
total sessions: 22,308
sessions where pools were closed: 879 (3.9%)
sessions with late openings: 1,539 (6.9%)

Closure reasons:
ClosedReason
Repairs                                                        109
Weather                                                         82
Other                                                           56
Category 2                                                      48
Lifeguards not on site                                          37
Repairs, Other                                                  29
Excessive turbidity                                             15
NYPD not on site                                                11
Weather, Other                                                   6
NYPD not on site, Weather                                        3
PEP not on site, Lifeguards not on site                          2
PEP not on site                                                  2
Lifeguards not on site, Repairs  

In [48]:
# === CHART 1 ===

print(f"CHART 1: POOL TYPES (Bar chart)\nNumber of pools per pool type\n")

# count pools by type from the pools dataset
pool_types = pools_clean['POOLTYPE'].value_counts().sort_values(ascending=False)
print(pool_types)

# export to csv
pool_types_df = pd.DataFrame({
    'pool_type': pool_types.index,
    'count': pool_types.values
})
pool_types_df.to_csv('data/chart1_pool_types.csv', index=False)
print(f"\nChart 1 data exported to: data/chart1_pool_types.csv")


CHART 1: POOL TYPES (Bar chart)
Number of pools per pool type

POOLTYPE
Intermediate             27
Wading                   23
Mini                     17
Olympic                  14
Large                     5
Diving                    2
Intermediate & Diving     1
Olympic & Diving          1
Name: count, dtype: int64

Chart 1 data exported to: data/chart1_pool_types.csv


In [205]:
# === CHART 2 ===

print(f"CHART 2: LARGEST AND SMALLEST POOLS (HorizontalBar chart)\nby cubic feet\n")

# got help from Cursor
# Extract pool dimensions and calculate cubic feet
def extract_pool_dimensions(size_str):
    """Extract length, width, depth from size string and calculate cubic feet"""
    if pd.isna(size_str) or size_str == 'None' or size_str == '':
        return None
    
    # Handle different size formats
    import re
    
    # Pattern for "length' x width' x depth'" format
    pattern = r"(\d+(?:\.\d+)?)'?\s*x\s*(\d+(?:\.\d+)?)'?\s*x\s*(\d+(?:\.\d+)?)'?"
    match = re.search(pattern, str(size_str))
    
    if match:
        length, width, depth = map(float, match.groups())
        return length * width * depth
    
    # Handle round pools (diameter format)
    round_pattern = r"(\d+(?:\.\d+)?)'?\s*\(round\)\s*x\s*(\d+(?:\.\d+)?)'?"
    round_match = re.search(round_pattern, str(size_str))
    if round_match:
        diameter, depth = map(float, round_match.groups())
        # Approximate area of circle * depth
        import math
        radius = diameter / 2
        area = math.pi * radius * radius
        return area * depth
    
    return None

# Apply to outdoor pools dataset
outdoor_clean['cubic_feet'] = outdoor_clean['Size'].apply(extract_pool_dimensions)

# Filter out pools without size data and sort by cubic feet
pools_with_size = outdoor_clean.dropna(subset=['cubic_feet']).copy()
pools_with_size = pools_with_size.sort_values('cubic_feet', ascending=False)
print(f'pools with size info: {len(pools_with_size)}\n')

# Get largest and smallest, arbitrary cut-off at 7 for aesthetic reasons
largest_pools = pools_with_size.head(7)
smallest_pools = pools_with_size.tail(7)

# Combine and sort for chart
chart2_data = pd.concat([largest_pools, smallest_pools]).sort_values('cubic_feet', ascending=False)

print("Largest pools:")
print(largest_pools[['Name', 'Borough_Name', 'cubic_feet']].to_string(index=False))

print("\nSmallest pools:")
print(smallest_pools[['Name', 'Borough_Name', 'cubic_feet']].to_string(index=False))

# Export for Chart.js
chart2_df = pd.DataFrame({
    'pool_name': chart2_data['Name'],
    'borough': chart2_data['Borough_Name'],
    'cubic_feet': chart2_data['cubic_feet'].round(0)
})
chart2_df.to_csv('data/chart2_pool_sizes.csv', index=False)
print(f"\nChart 2 data exported to: data/chart2_pool_sizes.csv")

CHART 2: LARGEST AND SMALLEST POOLS (HorizontalBar chart)
by cubic feet

pools with size info: 77

Largest pools:
        Name Borough_Name  cubic_feet
  Highbridge    Manhattan    374220.0
  Betsy Head     Brooklyn    231412.5
    McCarren     Brooklyn    228523.4
     Astoria       Queens    217800.0
    Red Hook     Brooklyn    171600.0
Crotona Pool        Bronx    158400.0
 Sunset Park     Brooklyn    146853.0

Smallest pools:
               Name Borough_Name  cubic_feet
             Haffen        Bronx  576.000000
          Claremont        Bronx  576.000000
    Commodore Barry     Brooklyn  576.000000
Douglass and DeGraw     Brooklyn  576.000000
        Fort Totten       Queens  471.238898
            Liberty       Queens  400.000000
             Fisher       Queens  400.000000

Chart 2 data exported to: data/chart2_pool_sizes.csv


In [92]:
# === CHART 3 ===

print(f"CHART 3: POOL ATTENDANCE NUMBERS (Horizontal Bar chart)\n")

# got help from Cursor
# Calculate average attendance per pool
attendance_by_pool = sessions_clean.groupby(['Pool', 'Borough_Name']).agg({
    'Attendance': 'mean'
}).reset_index()

# Sort by average attendance (descending)
attendance_by_pool = attendance_by_pool.sort_values('Attendance', ascending=False)

print("Top 10 pools by average attendance:")
print(attendance_by_pool.head(10)[['Pool', 'Borough_Name', 'Attendance']].to_string(index=False))

# Export for Chart.js
chart3_df = pd.DataFrame({
    'pool_name': attendance_by_pool['Pool'],
    'borough': attendance_by_pool['Borough_Name'],
    'avg_attendance': attendance_by_pool['Attendance'].round(0)
})
chart3_df.to_csv('data/chart3_attendance.csv', index=False)
print(f"\nChart 3 data exported to: data/chart3_attendance.csv")


CHART 3: POOL ATTENDANCE NUMBERS (Horizontal Bar chart)

Top 10 pools by average attendance:
              Pool Borough_Name  Attendance
      Astoria Pool       Queens  788.272727
McCarren Park Pool     Brooklyn  619.868534
  Sunset Park Pool     Brooklyn  381.389610
      Crotona Pool        Bronx  288.782511
   Highbridge Pool    Manhattan  261.695279
Hamilton Fish Pool    Manhattan  261.051613
   Kosciuszko Pool     Brooklyn  259.582589
     Red Hook Pool     Brooklyn  248.493333
       Fisher Pool       Queens  234.829322
     John Jay Pool    Manhattan  222.503226

Chart 3 data exported to: data/chart3_attendance.csv


In [144]:
len(attendance_by_pool)

51

In [69]:
# === CHART 4 ===

print(f"CHART 4: WEEKLY POOP INCIDENTS (Stacked Bar Chart)\n")

# got help from Cursor
# Filter for Category 1 and Category 2 incidents (poop incidents)
# Category 1 = 45 minute closure, Category 2 = 25.5 hour closure

# Create a copy for analysis
sessions_poop = sessions_clean.copy()

# Identify Category 1 incidents (in LateOpenReason)
sessions_poop['category_1'] = sessions_poop['LateOpenReason'].str.contains('Category 1', na=False).astype(int)

# Identify Category 2 incidents (in ClosedReason) 
sessions_poop['category_2'] = sessions_poop['ClosedReason'].str.contains('Category 2', na=False).astype(int)

# Add week number and year
sessions_poop['week'] = sessions_poop['ObservationDate'].dt.isocalendar().week
sessions_poop['year'] = sessions_poop['ObservationDate'].dt.year
sessions_poop['month'] = sessions_poop['ObservationDate'].dt.month

CHART 4: WEEKLY POOP INCIDENTS (Stacked Bar Chart)



In [70]:
sessions_poop.shape

(22308, 43)

In [77]:
# pools with no poop (both category_1 and category_2 are 0)
sessions_no_poop = sessions_poop[(sessions_poop['category_1'] == 0) & (sessions_poop['category_2'] == 0)].copy()
print(sessions_no_poop.shape)


(22253, 43)


In [76]:
print(f'number of sessions that had poop: {(len(sessions_poop) - len(sessions_no_poop))}')

number of sessions that had poop: 55


In [87]:
# Group by week and sum incidents
weekly_incidents = sessions_poop.groupby(['year', 'week', 'month']).agg({
    'category_1': 'sum',
    'category_2': 'sum'
}).reset_index()

print("Total Category 1 incidents:", sessions_poop['category_1'].sum())
print("Total Category 2 incidents:", sessions_poop['category_2'].sum())

# Filter for pool season (June-September)
weekly_incidents = weekly_incidents[weekly_incidents['month'].isin([6, 7, 8, 9])]

# Create week labels
weekly_incidents['week_label'] = (
    weekly_incidents['week'].astype(str) + ' (' +
    weekly_incidents['month'].astype(str) + ')'
)

print(weekly_incidents[['week_label', 'category_1', 'category_2']].to_string(index=False))

Total Category 1 incidents: 7
Total Category 2 incidents: 48
week_label  category_1  category_2
    26 (6)           0           0
    26 (7)           0           0
    27 (7)           1           0
    28 (7)           0           0
    29 (7)           0           0
    30 (7)           0           0
    31 (8)           0           0
    32 (8)           0           0
    33 (8)           0           0
    34 (8)           0           0
    35 (8)           0           0
    35 (9)           0           0
    36 (9)           0           0
    26 (6)           0           0
    26 (7)           0           0
    27 (7)           2           0
    28 (7)           0           0
    29 (7)           0           2
    30 (7)           0           2
    31 (7)           0           0
    31 (8)           0           1
    32 (8)           0           4
    33 (8)           0           0
    34 (8)           2           4
    35 (8)           0           2
    35 (9)           0       

In [88]:
weekly_incidents.head()

,year,week,month,category_1,category_2,week_label
0,2022,26,6,0,0,26 (6)
1,2022,26,7,0,0,26 (7)
2,2022,27,7,1,0,27 (7)
3,2022,28,7,0,0,28 (7)
4,2022,29,7,0,0,29 (7)


In [89]:
# Export for Chart.js
chart4_df = pd.DataFrame({
    'week': weekly_incidents['week_label'],
    'category_1': weekly_incidents['category_1'],
    'category_2': weekly_incidents['category_2'],
    'month': weekly_incidents['month'],
    'year': weekly_incidents['year']
})
chart4_df.to_csv('data/chart4_weekly_incidents.csv', index=False)
print(f"\nChart 4 data exported to: data/chart4_weekly_incidents.csv")


Chart 4 data exported to: data/chart4_weekly_incidents.csv


In [154]:
# === CHART 5 ===

print(f"CHART 5: TOP 10 MOST POOP INCIDENTS (Horizontal Bar)\n")

# Count total incidents per pool (Category 1 + Category 2)
pool_incidents = sessions_poop.groupby(['Pool', 'Borough_Name']).agg({
    'category_1': 'sum',
    'category_2': 'sum'
}).reset_index()
print(f'pool with incidents:{len(pool_incidents)}\n')

# Calculate total incidents
pool_incidents['total_incidents'] = pool_incidents['category_1'] + pool_incidents['category_2']

# Sort by total incidents (descending) and get top 10
top_10_offending = pool_incidents.sort_values('total_incidents', ascending=False).head(10)

print("Top 10 pools with most poop incidents:")
print(top_10_offending[['Pool', 'Borough_Name', 'category_1', 'category_2', 'total_incidents']].to_string(index=False))

CHART 5: TOP 10 MOST POOP INCIDENTS (Horizontal Bar)

pool_incidents:51
Top 10 pools with most poop incidents:
          Pool  Borough_Name  category_1  category_2  total_incidents
 Red Hook Pool      Brooklyn           0           9                9
  Crotona Pool         Bronx           0           5                5
  Astoria Pool        Queens           0           5                5
   Howard Pool      Brooklyn           0           4                4
    Mapes Pool         Bronx           1           3                4
    Faber Pool Staten Island           0           3                3
 Bushwick Pool      Brooklyn           1           2                3
  Liberty Pool        Queens           0           3                3
Claremont Pool         Bronx           0           2                2
   Fisher Pool        Queens           0           2                2


In [ ]:
# Export for Chart.js
chart5_df = pd.DataFrame({
    'pool_name': top_10_offending['Pool'],
    'borough': top_10_offending['Borough_Name'],
    'category_1': top_10_offending['category_1'],
    'category_2': top_10_offending['category_2'],
    'total_incidents': top_10_offending['total_incidents']
})
chart5_df.to_csv('data/chart5_top_offending_pools.csv', index=False)
print(f"\nChart 5 data exported to: data/chart5_top_offending_pools.csv")

In [202]:
# === CHART 6 ===

print(f"CHART 6: INCIDENTS BY POOL SIZE (Scatter Chart)\n")

# get pools that had incidents
print(f'pool_incidents:{len(pool_incidents)}')



CHART 6: INCIDENTS BY POOL SIZE (Scatter Chart)

pool_incidents:51


In [208]:
# exported these two files to csv to merge by hand
# some have more than one entry because of more than one pool (like a wading pool) at each location
pool_incidents.to_csv('data/chart6_incidents_by_pool_size.csv', index=False)
pools_with_size.to_csv('data/pools_with_size.csv', index=False)


Exported pool_incidents to data/chart6_incidents_by_pool_size.csv
